# import packages and important functions

In [1]:
#used to stop program under certain conditions
from sys import exit

#dataframes and numerical processes
import pandas as pd
import numpy as np

#importing database from file or online
import csv
from urllib import request
from sodapy import Socrata

# visualization
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import plotly.plotly as py

#for converting latitude and longitude into distance
from math import sin, cos, sqrt, atan2, radians

#Do not display warnings
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

#if value is NaN, return ifnan_value, otherwise return value
def if_null_value(value,if_nan_value):
    return (if_nan_value if pd.isnull(value) else value)

# If state is listed as "ZZ" or "XX", than zip codeis null.
# Fill with 1 if 'ZZ' (unknown location)
# or 2 if 'XX'. Not in the US 
# or 3 if 'AP' for Pacific armed forces
def fill_zip_code(state,zipcode):
    final_code = ''
    if not pd.isnull(zipcode):
        final_code = zipcode
    elif state == 'ZZ':
        final_code = 1
    elif state == 'XX':
        final_code = 2
    elif state == 'AP':
        final_code = 3
    else:
        print('ERROR: state: \'{}\' not recognized'.format(state))
        exit()
    return final_code


#combines strings, used to combine first and last names with spaces between.
def combine_strings(*strings):
    combine = ''
    for x in strings:
        if isinstance(x,str):
          combine += x + ' '
    if(combine == ''):
        return combine
    else:
        return combine[0:-1] 

#if latitude and longitude of zip code don't match the others of the city, replace them with
#mode. This is done in cases there mistake in the database
def replace_lat_and_lat(lat_long,median_lat_long,lat_wrong,long_wrong):
    if(lat_wrong or long_wrong):
        return lat_long
    else:
        return median_lat_long

#This function returns the city with the lowest distance based on longitude and latitude  
def smallest_distance_city(long,lat,long_lat_df):
    return long_lat_df[['Longitude','Latitude']].apply(lambda x:
                                                             (get_distance(x[0],x[1],long,lat)),
                                                             axis=1).idxmin()
    
#get distance of two longitude and latitude     
def get_distance(long1,lat1,long2,lat2):
    # approximate radius of earth in km
    R = 6373.0
    long1,lat1,long2,lat2  = radians(long1),radians(lat1),radians(long2),radians(lat2)
    dlong,dlat = long1 - long2,lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlong / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Create dataframe of population by zipcode
The population of a zip code is replaced with the sum of populations of zip codes in the same city. 
Considering that these zip codes will be geographically close, the city population is a better estimate of the population
of a health provider's cliental. The zip code is linked to a city and state using a second database (See below). This will also also fill many of empty populations zip codes with empty entries.

Source: https://simplemaps.com/data/us-zips 
accessed: November 7, 2018

In [2]:
def create_pop_dataframe():
    #population from the 2010 census
    filename = 'Zip_code_to_address/uszipsv1.4.csv'
    population_df = pd.read_csv(filename)[['zip','city','state_id','lat','lng','population']]
    population_df.rename(columns={'zip':'zipcode'},inplace=True)
      
    #summing over the population of zip codes in the same city
    #and replacing the population with it.This will also fill null values
    location_pop=population_df.groupby(['state_id','city'])['city','state_id','population'].sum().reset_index()

    #round the longitude and latitude to 1 digits then take the mode of a city. 
    #This is used to determine if there are any cities that the longitude and latitude 
    # are incorrect.
    population_df['lat'] = population_df['lat'].fillna(0)
    population_df['lng'] = population_df['lng'].fillna(0)
    location_pop['lat median'] = population_df.groupby(['state_id','city']).median().reset_index()['lat']    
    location_pop['lng median'] = population_df.groupby(['state_id','city']).median().reset_index()['lng']
    
    population_df=population_df.merge(location_pop,how='left',on=['state_id','city'])
    population_df.rename(columns={'population_x':'population','population_y':'population sum'},inplace=True)

    #if longitude and latitude are too far from median for the city, it is most likely wrong.
    #Replace with median.
    population_df['lat match']=population_df[['lat','lat median']].apply(lambda x:(abs(x[0]-x[1])<2) ,axis=1)
    population_df['lng match']=population_df[['lng','lng median']].apply(lambda x:(abs(x[0]-x[1])<2) ,axis=1)
    population_df['lat'] = population_df[['lat','lat median','lat match','lng match']].apply(lambda x:replace_lat_and_lat(x[0],x[1],x[2],x[3]),axis=1)    
    population_df['lng'] = population_df[['lng','lng median','lat match','lng match']].apply(lambda x:replace_lat_and_lat(x[0],x[1],x[2],x[3]),axis=1)  
    population_df.drop(columns=['lat match','lng match','lat median','lng median'],axis=1,inplace=True)  
    
    
    #if the population sum is zero. Do not use it because the population of all the zip code in that city are null
    population_df['population'] =  population_df[['population','population sum']].apply(lambda x:
                                                                                          x[1] if x[1] != 0 
                                                                                          else x[0],
                                                                                         axis=1)
    population_df.drop(columns='population sum',axis=1,inplace=True)
    
        
    return population_df,location_pop
    
if __name__ == '__main__':
    population_df,location_pop=create_pop_dataframe()

In [41]:
#display map
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
           [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]
data = [ dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = population_df['lng'],
    lat = population_df['lat'],
    text = population_df['city'] + ',' + population_df['state_id'],
    mode = 'markers',
    marker = dict(
        size = 8,
        opacity = 0.8,
        reversescale = True,
        autocolorscale = False,
        symbol = 'cicle',
        line = dict(
            width=1,
            color='rgba(102, 102, 102)'
        ),
        colorscale = scl,
        cmin = 0,
        color = population_df['population'],
        cmax = population_df['population'].max(),
        colorbar=dict(
            ttitle="city population"
        )
    ))]
layout = dict(
    title = 'City population',
    colorbar = True,
    geo = dict(
        scope='usa',
        projection=dict( type='albers usa' ),
        showland = True,
        landcolor = "rgb(250, 250, 250)",
        subunitcolor = "rgb(217, 217, 217)",
        countrycolor = "rgb(217, 217, 217)",
        countrywidth = 0.5,
        subunitwidth = 0.5
    ),
)

fig = dict( data=data, layout=layout)
py.iplot( fig, validate=False, filename='population')

# Temperature Data in celsius
Temperature data is used to assign a temperature to all the cities in the population database. The data temperatures only has the temperature of about 30 cities. Temperature all the rest set by the temperature of the closest data point in the temperature data base. Unfortunately this creates temperature clusters rather than continues changes in temperature. Currently looking for a better solution.

Source: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data
Acessed: November 28,2018

In [4]:
def add_temperature_to_dataframe(population_df):
    #read and create temperature dataframe
    filename = 'kaggle_temperature/GlobalLandTemperaturesByCity.csv'
    temperature_df = pd.read_csv(filename)
    
    #Only cities in the US in 2013
    temperature_df = temperature_df[temperature_df['Country'] == 'United States']
    temperature_df['dt']=pd.to_datetime(temperature_df['dt'])
    temperature_df = temperature_df[temperature_df['dt'].dt.year == 2013]
    
    #Group by city to find the high, low, and mean of a city in 2013
    temperature_clean = temperature_df.groupby(['City']).min()[['Latitude','Longitude','AverageTemperature']]
    temperature_clean.rename(columns={'AverageTemperature':'Min Temp'},inplace=True)
    temperature_clean['Max Temp'] = temperature_df.groupby(['City']).max()['AverageTemperature']
    temperature_clean['Mean Temp'] = temperature_df.groupby(['City']).mean()['AverageTemperature']                            
    
    #converting Latitude and latitude into floats
    temperature_clean['Longitude'] =-temperature_clean['Longitude'].apply(lambda x:x[:-1]).astype(float)
    temperature_clean['Latitude'] =temperature_clean['Latitude'].apply(lambda x:x[:-1]).astype(float)
    
    city_min = population_df[['lng','lat']].apply(lambda x:smallest_distance_city(x[0],x[1],temperature_clean),axis=1)
    population_df['Mean Temp'] = pd.Series([temperature_clean.loc[i]['Mean Temp']for i in city_min])
    population_df['Min Temp'] = pd.Series([temperature_clean.loc[i]['Min Temp']for i in city_min])
    population_df['Max Temp'] = pd.Series([temperature_clean.loc[i]['Max Temp']for i in city_min])
    
    return population_df,temperature_clean
if __name__ == '__main__':
    population_df, temperature_df= add_temperature_to_dataframe(population_df)  
population_df.head()

,zipcode,city,state_id,lat,lng,population,Mean Temp,Min Temp,Max Temp
0,501,Holtsville,NY,40.8133,-73.0476,13261.0,12.329111,0.389,24.519
1,544,Holtsville,NY,40.8133,-73.0476,13261.0,12.329111,0.389,24.519
2,601,Adjuntas,PR,18.1800,-66.7522,18570.0,24.440111,17.764,28.830
3,602,Aguada,PR,18.3607,-67.1752,41520.0,24.440111,17.764,28.830
4,603,Aguadilla,PR,18.4544,-67.1220,54689.0,24.440111,17.764,28.830


# display temperature map

In [44]:
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
           [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]
data = [ dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = temperature_df['Longitude'],
    lat = temperature_df['Latitude'],
    text = list(temperature_df.index),
    mode = 'markers',
    marker = dict(
        size = 8,
        opacity = 0.8,
        reversescale = True,
        autocolorscale = False,
        symbol = 'cicle',
        line = dict(
            width=1,
            color='rgba(102, 102, 102)'
        ),
        colorscale = scl,
        cmin = 0,
        color = temperature_df['mean Temp'],
        cmax = temperature_df['mean Temp'].max(),
        colorbar=dict(
            ttitle="city temperature"
        )
    ))]
layout = dict(
    title = 'City temperature',
    colorbar = True,
    geo = dict(
        scope='usa',
        projection=dict( type='albers usa' ),
        showland = True,
        landcolor = "rgb(244, 146, 66)",
        subunitcolor = "rgb(217, 217, 217)",
        countrycolor = "rgb(217, 217, 217)",
        countrywidth = 0.5,
        subunitwidth = 0.5
    ),
)

fig = dict( data=data, layout=layout)
py.iplot( fig, validate=False, filename='temperature')

# display temperature  of population database

In [46]:
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
           [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]
data = [ dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = population_df['lng'],
    lat = population_df['lat'],
    text = list(population_df.index),
    mode = 'markers',
    marker = dict(
        size = 8,
        opacity = 0.8,
        reversescale = True,
        autocolorscale = False,
        symbol = 'cicle',
        line = dict(
            width=1,
            color='rgba(102, 102, 102)'
        ),
        colorscale = scl,
        cmin = 0,
        color = population_df['Mean Temp'],
        cmax = population_df['Mean Temp'].max(),
        colorbar=dict(
            ttitle="city temperature"
        )
    ))]
layout = dict(
    title = 'City temperature',
    colorbar = True,
    geo = dict(
        scope='usa',
        projection=dict( type='albers usa' ),
        showland = True,
        landcolor = "rgb(244, 146, 66)",
        subunitcolor = "rgb(217, 217, 217)",
        countrycolor = "rgb(217, 217, 217)",
        countrywidth = 0.5,
        subunitwidth = 0.5
    ),
)

fig = dict( data=data, layout=layout)
py.iplot( fig, validate=False, filename='temperature')

# Create dataframe for API

Here a dataframe for the percentage and amount of opioid prescribed using Medicare Part D by health care providers. The data is cleaned and using the previous dataset, the population of the area around each healthcare provider is set.

Source: https://data.cms.gov/Medicare-Claims/Medicare-Part-D-Opioid-Prescriber-Summary-File-201/yb2j-f3fp 
first access: Oct 30, 2018

In [47]:
def create_opioid_dataframe(population_df):

    WEBSITE = 'data.cms.gov'
    DATASET_ID = 'aksg-4qws'

    client = Socrata(WEBSITE,None)
    results = client.get(DATASET_ID,content_type='json',limit=1500000)
    opioid_df = pd.DataFrame.from_records(results)[['npi',
                                                    'nppes_provider_first_name',
                                                    'nppes_provider_last_name',
                                                    'nppes_provider_zip_code',
                                                    'nppes_provider_state',
                                                    'specialty_description',
                                                    'total_claim_count',
                                                    'opioid_claim_count',
                                                    'percent_opioid_claims']]


    #change name of columns
    opioid_df.columns =['doctor id',
                        'first name',
                        'last name',                    
                        'zip code',                       
                        'state',
                        'specialty description',
                        'total claims',
                        'opioid claims',
                        'percent opioid claims']
    #set doctor id to the index
    opioid_df.set_index('doctor id',inplace=True)
    #combine first and last name into a single column
    opioid_df['doctor name']=opioid_df[['first name','last name']].apply(lambda x:combine_strings(*x),axis=1)
    opioid_df.drop(labels=['first name','last name'],axis=1,inplace=True)

    #Unknown location (state listed as ZZ) gets a zipcode of 0
    #Not in the US (state listed as XX) get a zipcode of 1
    opioid_df['zip code'] = opioid_df[['state','zip code']].apply(lambda x:fill_zip_code(x[0],x[1]),axis=1)

    #setting specialty description index
    doc_spec = set(list(opioid_df['specialty description']))
    spec_compreh=((x,counter) for counter, x in enumerate(doc_spec))
    spec_df=pd.DataFrame(spec_compreh,columns=['specialty description','specialty index'])
    opioid_df = opioid_df.merge(spec_df,how='left',on='specialty description')

    #merging with population data
    population_df['zipcode'] = population_df['zipcode'].astype(int)
    opioid_df['zip code'] = opioid_df['zip code'].astype(int)
    opioid_df = opioid_df.merge(population_df,how='left',left_on='zip code',right_on='zipcode')
    opioid_df.drop('zipcode',axis=1,inplace=True)
    opioid_df.drop('state',axis=1,inplace=True)

    #Droping doctors with nan or zero claims.
    #converting fraction of opioid claims to a percentage
    opioid_df['opioid claims']= opioid_df['opioid claims'].apply(lambda x: if_null_value(x,0))
    opioid_df['opioid claims']= opioid_df['opioid claims'].astype(int)
    opioid_df['percent opioid claims']=opioid_df['percent opioid claims'].apply(lambda x: if_null_value(x,0))
    opioid_df['percent opioid claims']=opioid_df['percent opioid claims'].astype(float)
    opioid_df['total claims']= opioid_df['total claims'].astype(int)
    opioid_reduced=opioid_df[opioid_df['opioid claims'] != 0.0]


    # Quick analyze of data and outlier detection
    print('# health providers: {}'
          .format(opioid_df['doctor name'].count()))
    print('# health providers who prescribe opioid: {}'
          .format(opioid_reduced['doctor name'].count()))
    print('% health providers that prescribe opioids: {:.2f}%'.
         format(opioid_reduced['doctor name'].count()/opioid_df['doctor name'].count()*100))
    print('# specialities: {}'.format(spec_df['specialty index'].count()))
    print('# specialities that prescribe opioid: {}'
          .format(opioid_reduced['specialty index'].nunique()))
    print('Max % prescriptions that are opioids: {:.2f}%'
          .format(opioid_reduced['percent opioid claims'].max()))
    print('Min % prescriptions that are opioids(other than zero): {:.2f}%'
          .format(opioid_reduced['percent opioid claims'].min()))
    print('Missing population for {} ({:.2f}%) entries'.
          format(opioid_df[(opioid_df['population'].isnull())]['zip code'].count()
              ,opioid_df[(opioid_df['population'].isnull()) ]['zip code'].count()/opioid_df['zip code'].count()*100))

    #Get rid of the remaining null values
    opioid_df=opioid_df[opioid_df['population'].notnull()]
    opioid_reduced=opioid_reduced[opioid_reduced['population'].notnull()]
    return opioid_df,opioid_reduced
if __name__ == '__main__':
    opioid_df,opioid_reduced = create_opioid_dataframe(population_df)

# health providers: 1049326
# health providers who prescribe opioid: 496744
% health providers that prescribe opioids: 47.34%
# specialities: 246
# specialities that prescribe opioid: 169
Max % prescriptions that are opioids: 100.00%
Min % prescriptions that are opioids(other than zero): 0.03%
Missing population for 1820 (0.17%) entries


In [48]:
opioid_df

,zip code,specialty description,total claims,opioid claims,percent opioid claims,doctor name,specialty index,city,state_id,lat,lng,population,Mean Temp,min Temp,max Temp
0,10016,Pharmacist,18,0,0.000000,10 PARK PHARMACY INC.,126,New York,NY,40.7451,-73.9783,1577385.0,12.163889,-1.365,24.722
1,91303,Clinic/Center,84,0,0.000000,"101 FAMILY MEDICAL GROUP,A PROFESSIONAL CORP",146,Canoga Park,CA,34.1979,-118.6016,77086.0,18.120667,8.813,25.090
2,33014,Internal Medicine,29,0,0.000000,"154ST STREET MEDICAL PLAZA, INC.",154,Hialeah,FL,25.9043,-80.3029,347944.0,24.440111,17.764,28.830
3,33180,Urology,27,0,0.000000,21ST CENTURY ONCOLOGY LLC,221,Miami,FL,25.9607,-80.1420,1723766.0,24.440111,17.764,28.830
4,98225,Dentist,28,0,0.000000,3 SUNS INVESTMENT INC,78,Bellingham,WA,48.7531,-122.5015,117728.0,9.946444,0.980,17.033
5,4046,Dentist,17,0,0.000000,"3RD GENERATION DENTISTRY, PC",78,Kennebunkport,ME,43.4203,-70.4920,7496.0,10.375667,-4.140,23.629
6,95351,Psychiatry & Neurology,13,0,0.000000,"4 N PSYCH, INC.",225,Modesto,CA,37.6233,-120.9963,255354.0,15.020889,3.579,24.461
7,70123,Ophthalmology,29,0,0.000000,"4 YOUR EYES, LLC",185,New Orleans,LA,29.9509,-90.2051,382333.0,21.995222,13.746,28.306
8,11104,Family Practice,55,0,0.000000,43RD ST FAM PRACT PLLC,100,Sunnyside,NY,40.7446,-73.9203,27232.0,12.163889,-1.365,24.722
9,90211,Internal Medicine,48,0,0.000000,"8920 MEDICAL ASSOCIATES, INC.",154,Beverly Hills,CA,34.0650,-118.3830,41730.0,18.120667,8.813,25.090
